In [ ]:
# See https://docs.kedro.org/en/stable/notebooks_and_ipython/kedro_and_notebooks.html
# %load_ext kedro.ipython

In [ ]:
# catalog.list()

In [1]:
# <project_root>/register_prefect_flow.py
import click

[10/29/24 11:37:28] INFO     Using                                                                  ]8;id=744898;file:///home/guillaume/Workspace/kedro-dagster/.venv/lib/python3.12/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=794165;file:///home/guillaume/Workspace/kedro-dagster/.venv/lib/python3.12/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/home/guillaume/Workspace/kedro-dagster/.venv/lib/python3.12/site-pac                
                             kages/kedro/framework/project/rich_logging.yml' as logging                            
                             configuration.                                                                        

In [5]:
from dagster import get_dagster_logger


def kedro_init(
    pipeline_name: str,
    project_path: Path,
    env: str,
):
    """
    Initializes a Kedro session and returns the DataCatalog and KedroSession.


    """
    # bootstrap project within task / flow scope

    logger = get_dagster_logger()
    logger.info("Bootstrapping project")
    bootstrap_project(project_path)

    session = KedroSession.create(
        project_path=project_path,
        env=env,
    )
    # Note that for logging inside a Prefect task logger is used.
    logger.info("Session created with ID %s", session.session_id)
    pipeline = pipelines.get(pipeline_name)
    logger.info("Loading context...")
    context = session.load_context()
    catalog = context.catalog
    logger.info("Registering datasets...")
    unregistered_ds = pipeline.datasets() - set(catalog.list())
    for ds_name in unregistered_ds:
        catalog.add(ds_name, MemoryDataset())
    return {"catalog": catalog, "sess_id": session.session_id}

In [22]:
execution_config = kedro_init(
    pipeline_name="data_processing", project_path="../", env=None
)

[10/29/24 18:29:46] INFO     Bootstrapping project                                                 ]8;id=300025;file:///tmp/ipykernel_938598/2989318381.py\2989318381.py]8;;\:]8;id=323317;file:///tmp/ipykernel_938598/2989318381.py#16\16]8;;\

                    INFO     Session created with ID 2024-10-29T17.29.46.521Z                      ]8;id=46701;file:///tmp/ipykernel_938598/2989318381.py\2989318381.py]8;;\:]8;id=174074;file:///tmp/ipykernel_938598/2989318381.py#24\24]8;;\

                    INFO     Loading context...                                                    ]8;id=527849;file:///tmp/ipykernel_938598/2989318381.py\2989318381.py]8;;\:]8;id=538230;file:///tmp/ipykernel_938598/2989318381.py#26\26]8;;\

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=708771;file:///home/guillaume/Workspace/kedro-dagster/.venv/lib/python3.12/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=800207;file:///home/guillaume/Workspace/kedro-dagster/.venv/lib/python3.12/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

                    INFO     Registering datasets...                                               ]8;id=327671;file:///tmp/ipykernel_938598/2989318381.py\2989318381.py]8;;\:]8;id=804976;file:///tmp/ipykernel_938598/2989318381.py#29\29]8;;\

In [26]:
catalog = execution_config["catalog"]
catalog.list()


[
    'companies',
    'reviews',
    'shuttles',
    'preprocessed_companies',
    'preprocessed_shuttles',
    'model_input_table',
    'regressor',
    'parameters',
    'params:model_options',
    'params:model_options.test_size',
    'params:model_options.random_state',
    'params:model_options.features'
]

In [30]:
pipelines["__default__"].nodes


[
    Node(preprocess_companies, 'companies', 'preprocessed_companies', 'preprocess_companies_node'),
    Node(preprocess_shuttles, 'shuttles', 'preprocessed_shuttles', 'preprocess_shuttles_node'),
    Node(create_model_input_table, ['preprocessed_shuttles', 'preprocessed_companies', 'reviews'], 'model_input_table', 'create_model_input_table_node'),
    Node(split_data, ['model_input_table', 'params:model_options'], ['X_train', 'X_test', 'y_train', 'y_test'], 'split_data_node'),
    Node(train_model, ['X_train', 'y_train'], 'regressor', 'train_model_node'),
    Node(evaluate_model, ['regressor', 'X_test', 'y_test'], None, 'evaluate_model_node')
]

In [25]:
from kedro.io import DataCatalog, MemoryDataset

memory_assets = {
    asset_name: asset
    for asset_name in catalog.list()
    if isinstance(asset, MemoryDataset)
}
memory_assets

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2                                                                                              │
│   3 memory_assets = {                                                                            │
│   4 │   asset_name: asset                                                                        │
│ ❱ 5 │   for asset_name, asset in catalog.items()                                                 │
│   6 │   if isinstance(asset, MemoryDataset)                                                      │
│   7 }                                                                                            │
│   8 memory_assets                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'DataCatalog' object has no attribute 'items'

In [20]:
from pydantic import BaseModel


# Define the first Pydantic model
class ModelA(BaseModel):
    field1: str
    field2: int


# Define the second Pydantic model
class ModelB(BaseModel):
    field3: float
    field4: bool


class CombinedModel(ModelA, ModelB):
    pass


# Example usage
combined_instance = CombinedModel(field1="example", field2=42, field3=3.14, field4=True)

# Print the instance of the combined model
print(combined_instance)

field3=3.14 field4=True field1='example' field2=42


In [27]:
combined_instance.__annotations__

{'field1': <class 'str'>, 'field2': <class 'int'>}

In [ ]:
def init_kedro_tasks_by_execution_layer(
    pipeline_name: str,
    execution_config: Union[None, Dict[str, Union[DataCatalog, str]]] = None,
) -> List[List[Callable]]:
    """
    Inits the Kedro tasks ordered topologically in groups, which implies that an earlier group
    is the dependency of later one.

    Args:
        pipeline_name (str): The pipeline name to execute
        execution_config (Union[None, Dict[str, Union[DataCatalog, str]]], optional):
        The required execution config for each node. Defaults to None.

    Returns:
        List[List[Callable]]: A list of topologically ordered task groups
    """

    pipeline = pipelines.get(pipeline_name)

    execution_layers = []

    # Return a list of the pipeline nodes in topologically ordered groups,
    #  i.e. if node A needs to be run before node B, it will appear in an
    #  earlier group.
    for layer in pipeline.grouped_nodes:
        execution_layer = []
        for node in layer:
            # Use a function for task instantiation which avoids duplication of
            # tasks
            task = instantiate_task(node, execution_config)
            execution_layer.append(task)
        execution_layers.append(execution_layer)

    return execution_layers

In [ ]:
@flow(name="my_flow")
def my_flow(pipeline_name: str, env: str):
    logger = get_dagster_logger()  # Use context?
    project_path = Path.cwd()

    metadata = bootstrap_project(project_path)
    logger.info("Project name: %s", metadata.project_name)

    logger.info("Initializing Kedro...")
    execution_config = kedro_init(
        pipeline_name=pipeline_name, project_path=project_path, env=env
    )

    logger.info("Building execution layers...")
    execution_layers = init_kedro_tasks_by_execution_layer(
        pipeline_name, execution_config
    )

    for layer in execution_layers:
        logger.info("Running layer...")
        for node_task in layer:
            logger.info("Running node...")
            node_task()

In [ ]:
@click.command()
@click.option("-p", "--pipeline", "pipeline_name", default="__default__")
@click.option("--env", "-e", type=str, default="base")
@click.option("--deployment_name", "deployment_name", default="example")
@click.option("--work_pool_name", "work_pool_name", default="default")
@click.option("--work_queue_name", "work_queue_name", default="default")
@click.option("--version", "version", default="1.0")
def dagster_deploy(
    pipeline_name, env, deployment_name, work_pool_name, work_queue_name, version
):
    """Register a Kedro pipeline as a Dagster job."""

    # Pipeline name to execute
    pipeline_name = pipeline_name or "__default__"

    # Use standard deployment configuration for local execution. If you require a different
    # infrastructure, check the API docs for Deployments at: https://docs.prefect.io/latest/api-ref/prefect/deployments/
    deployment = Deployment.build_from_flow(
        flow=my_flow,
        name=deployment_name,
        path=str(Path.cwd()),
        version=version,
        parameters={
            "pipeline_name": pipeline_name,
            "env": env,
        },
        infra_overrides={"env": {"PREFECT_LOGGING_LEVEL": "DEBUG"}},
        work_pool_name=work_pool_name,
        work_queue_name=work_queue_name,
    )

    deployment.apply()